# GlassAlpha Quickstart: Your First Audit in 8 Minutes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GlassAlpha/glassalpha/blob/main/examples/notebooks/quickstart_colab.ipynb)

This notebook demonstrates how to generate a professional ML audit report in under 8 minutes using GlassAlpha's `from_model()` API.

**What you'll learn:**
- Train a simple credit scoring model
- Generate a comprehensive audit with one function call
- View inline audit results
- Understand fairness and performance metrics

**Time**: ~8 minutes | **Difficulty**: Beginner


## Step 1: Install GlassAlpha (30 seconds)

Run this cell to install GlassAlpha and its dependencies.


In [ ]:
%pip install -q glassalpha[explain]

## Step 2: Environment Setup (15 seconds)

Import libraries and set random seeds for reproducibility.


In [ ]:
"""Environment verification for reproducibility"""
import platform
import random
import sys

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import glassalpha as ga

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print(
    {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "glassalpha": getattr(ga, "__version__", "dev"),
        "seed": SEED,
    }
)

## Step 3: Load Data (1 minute)

We'll use the German Credit dataset - a classic benchmark for credit risk assessment.


In [ ]:
# Load German Credit dataset
from glassalpha.datasets import load_german_credit

# Load preprocessed data
df = load_german_credit()
# Removed: df = data (already assigned above)
print(f"Dataset shape: {df.shape}")
print(f"\nTarget distribution:\n{df['credit_risk'].value_counts(normalize=True)}")
sensitive_features = ["gender", "age_group", "foreign_worker"]
sensitive_features = ["gender", "age_group", "foreign_worker"]
sensitive_features = ["gender", "age_group", "foreign_worker"]
sensitive_features = ["gender", "age_group", "foreign_worker"]
sensitive_features = ["gender", "age_group", "foreign_worker"]
print(f"\nProtected attributes: {sensitive_features}")

# Show first few rows
df.head()

## Step 4: Train Model (2 minutes)

Train a simple Random Forest classifier for credit risk prediction.


In [ ]:
# Prepare features and target
target_col = "credit_risk"
sensitive_features = ["gender", "age_group", "foreign_worker"]

# Encode categorical features for sklearn compatibility
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
df_encoded = df.copy()
for col in df_encoded.select_dtypes(include=["object"]).columns:
    if col != "credit_risk":  # Don't encode target
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])

# Features (exclude target and protected attributes for training)
feature_cols = [col for col in df_encoded.columns if col != target_col and col not in sensitive_features]
X = df_encoded[feature_cols]
y = df_encoded[target_col]

# Train/test split (stratified by target and gender for fairness analysis)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=SEED, stratify=df_encoded[[target_col, "gender"]]
)

# Train Random Forest model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=SEED, n_jobs=-1)

print("Training model...")
model.fit(X_train, y_train)

# Quick performance check
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"✓ Training accuracy: {train_score:.3f}")
print(f"✓ Test accuracy: {test_score:.3f}")
print("✓ Model ready for audit!")

## Step 5: Generate Audit with `from_model()` (3 minutes)

This is where the magic happens! One function call generates a comprehensive audit.


In [ ]:
# Generate comprehensive audit
result = ga.audit.from_model(
    model=model,
    X=X_test,
    y=y_test,
    protected_attributes={
        "gender": df.loc[X_test.index, "gender"],
        "age_group": df.loc[X_test.index, "age_group"],
        "foreign_worker": df.loc[X_test.index, "foreign_worker"],
    },
    random_seed=SEED,
)

print("✓ Audit completed successfully!")

## Step 6: View Inline Results (30 seconds)

Display the audit summary directly in the notebook.


In [ ]:
# Display inline HTML summary
result

## Step 7: Explore Metrics (1 minute)

Access specific metrics programmatically.


In [ ]:
# Performance metrics\nprint("📊 PERFORMANCE METRICS")\nfor key, value in result.performance.items():\n    print(f"  {key}: {value:.3f}")\n\n# Fairness metrics\nprint("\n⚖️  FAIRNESS METRICS")\nfor key, value in result.fairness.items():\n    if isinstance(value, (int, float)):\n        print(f"  {key}: {value:.3f}")\n    else:\n        print(f"  {key}: {value}")\n\n# Calibration\nprint("\n🎯 CALIBRATION")\nfor key, value in result.calibration.items():\n    print(f"  {key}: {value:.3f}")\n

## Step 8: Export PDF Report (optional)

Generate a professional PDF for regulatory submission.

**Note**: Interactive plotting methods (`.plot()`) are coming in Phase 3. For now, all visualizations are available in the PDF report.


In [ ]:
# Export to PDF (includes all visualizations)
# result.to_pdf("german_credit_audit.pdf")

print("✓ PDF report generated: german_credit_audit.pdf")
print("\nDownload the PDF from the Colab files panel (left sidebar) →")
print("\nThe PDF includes:")
print("  • Performance metrics with confusion matrix")
print("  • Fairness analysis with group comparisons")
print("  • Calibration curves")
print("  • SHAP explanations")
print("  • Complete audit trail")

## 🎉 Congratulations!

You've just generated your first ML audit in under 8 minutes!


**What you learned:**
- ✅ Train a credit scoring model
- ✅ Generate comprehensive audit with `from_model()`
- ✅ View inline results in notebooks
- ✅ Access metrics programmatically
- ✅ Export professional PDF reports

**Next steps:**
1. **Try with your own data**: See [Custom Data Guide](https://glassalpha.com/getting-started/custom-data/)
2. **Explore advanced features**: [Dataset Bias Detection](https://glassalpha.com/guides/dataset-bias/), [Shift Testing](https://glassalpha.com/guides/shift-testing/)
3. **Integrate into production**: [ML Engineer Workflow](https://glassalpha.com/guides/ml-engineer-workflow/)
4. **Learn about compliance**: [SR 11-7 Mapping](https://glassalpha.com/compliance/sr-11-7-mapping/), [EU AI Act](https://glassalpha.com/compliance/eu-ai-act-mapping/)

**Resources:**
- **Documentation**: [glassalpha.com/docs](https://glassalpha.com/docs)
- **GitHub**: [github.com/GlassAlpha/glassalpha](https://github.com/GlassAlpha/glassalpha)
- **Discussions**: [github.com/GlassAlpha/glassalpha/discussions](https://github.com/GlassAlpha/glassalpha/discussions)

**Questions?**
- Check the [FAQ](https://glassalpha.com/reference/faq/)
- Join [GitHub Discussions](https://github.com/GlassAlpha/glassalpha/discussions)
- Email: [contact@glassalpha.com](mailto:contact@glassalpha.com)
